## Example local import

This notebook will auto-import recordings from the local disk

In [ ]:
# Define some functions

import lbdb
import kachery as ka
import labbox_ephys as le
import hither
import numpy as np
import json

def geom_from_recording(recording):
    channel_ids = recording.get_channel_ids()
    location0 = recording.get_channel_property(channel_ids[0], 'location')
    nd = len(location0)
    M = len(channel_ids)
    geom = np.zeros((M, nd))
    for ii in range(len(channel_ids)):
        location_ii = recording.get_channel_property(channel_ids[ii], 'location')
        geom[ii, :] = list(location_ii)
    return geom

def import_recording(*, recording_id, recording):
    print(f'Importing {recording_id}')
    doc = dict(
        recording_id=recording_id,
        recording=recording.object(),
        sampling_frequency=recording.get_sampling_frequency(),
        channel_ids=recording.get_channel_ids(),
        channel_groups=recording.get_channel_groups(),
        geom=geom_from_recording(recording).tolist(),
        num_frames=recording.get_num_frames()
    )
    le.dbcollection('recordings').replace_one(dict(recording_id=recording_id), doc, upsert=True)

In [ ]:
# Auto import local recordings

def auto_import_recordings(dirpath, prefix):
    print(f'Checking {dirpath}')
    x = ka.read_dir(dirpath)
    for dirname, dd in x['dirs'].items():
        if le.LabboxEphysRecordingExtractor.can_load_dir(dirpath + '/' + dirname):
            recording = LabboxEphysRecordingExtractor(dirpath + '/' + dirname)
            import_recording(recording_id=f'{prefix}{dirname}', recording=recording)
        else:
            auto_import_recordings(dirpath + '/' + dirname, prefix=prefix + dirname + '/')

auto_import_recordings(dirpath='/data/local-data', prefix='local:')

In [ ]:
# List all of the recordings in the database

recordings = le.dbcollection('recordings').find(dict())
for rec in recordings:
    print(rec['recording_id'])

In [ ]:
# Write a toy example recording for testing
# import spikeextractors as se
# from spikeforest2_utils import MdaRecordingExtractor
# R, S = se.example_datasets.toy_example()
# MdaRecordingExtractor.write_recording(recording=R, save_path='toy_example1')